In [2]:
import os
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F
from PIL import Image
from matplotlib.ticker import FormatStrFormatter
import torchvision.models as models

import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

from collections import OrderedDict
import torch.optim.lr_scheduler as lr_scheduler

import io
import pathlib
from torch.autograd import Variable
import seaborn as sns

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

class_names = train_image_datasets.classes

images, labels = next(iter(train_loader))
rand_idx = np.random.randint(len(images))
# print(rand_idx)
print("label: {}, class: {}, name: {}".format(labels[rand_idx].item(),
                                               class_names[labels[rand_idx].item()],
                                               cat_to_name[class_names[labels[rand_idx].item()]]))


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# define training and test data directories
data_dir = 'C:/Users/Sumit Anjna/Desktop/flower_data/'
train_dir = os.path.join(data_dir, 'train/')
valid_dir = os.path.join(data_dir, 'valid/')
test_dir = os.path.join(data_dir, 'test/')

In [ ]:
# TODO: Define your transforms for the training and validation sets
train_transforms = transforms.Compose([
                                      transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))])

valid_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                      transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))])

#test_transforms = transforms.Compose([transforms.ToTensor(),
                                      #transforms.RandomHorizontalFlip(),
                                      #transforms.RandomRotation(10),
                                      #transforms.Resize(224),
                                     # transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))])


train_image_datasets = datasets.ImageFolder(train_dir,transform=train_transforms)
valid_image_datasets = datasets.ImageFolder(valid_dir,transform=valid_transforms)
#test_image_datasets = datasets.ImageFolder(valid_dir,transform=test_transforms)


batch_size = 4
num_workers = 4



train_loader = torch.utils.data.DataLoader(train_image_datasets, batch_size=batch_size,shuffle=True,
                                           num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(valid_image_datasets,  batch_size=batch_size,shuffle=True,
                                           num_workers=num_workers)
#test_loader = torch.utils.data.DataLoader(test_image_datasets, batch_size=batch_size,shuffle=True,
                                           #num_workers=num_workers)

In [ ]:
model = models.densenet161(pretrained=True)

for param in model.parameters():
    param.requires_grad =False
model

In [ ]:
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                                ('fc1', nn.Linear(2208,512)),
                                ('bn1', nn.BatchNorm1d(512)),
                                ('relu1', nn.ReLU()),
                                ('fc2', nn.Linear(512,102)),
                                ('output', nn.LogSoftmax(dim=1))
                                ]))

model.classifier = classifier

In [ ]:
import torch.optim.lr_scheduler as lr_scheduler

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[20], gamma=0.1)
#scheduler = lr_scheduler.CosineAnnealingLR(optimizer,30, eta_min=0.00001 )
#scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, min_lr=0.00001)

In [ ]:
model.to(device)

In [ ]:
n_epochs = 35

valid_loss_min = np.Inf 

for epoch in range(1, n_epochs+1):
    
    train_loss = 0.0
    valid_loss = 0.0
    accuracy = 0.0
    scheduler.step(valid_loss)

    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        
        
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        
        output = model(data)
        
        loss = criterion(output, target)
        
        loss.backward()
        
        optimizer.step()
    
        train_loss += loss.item()*data.size(0)
        
    model.eval()
    for batch_idx, (data, target) in enumerate(valid_loader):
        

        data, target = data.to(device), target.to(device)
        
        output = model(data)
        
        loss = criterion(output, target)
         
        valid_loss += loss.item()*data.size(0)
        
        ps = torch.exp(output)
        _, top_class = ps.topk(1, dim=1)
        equals = top_class == target.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()*data.size(0)
    
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    acc = accuracy/len(valid_loader.dataset) 
        
     
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tAccuracy: {:.3f}'.format(
        epoch, train_loss, valid_loss, acc))
    
    
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_augmented.pt')
        valid_loss_min = valid_loss

In [ ]:
model.class_to_idx = train_image_datasets.class_to_idx
model.cpu
checkpoint = {'input_size': 2208,
              'output_size': 102,
              'epochs': n_epochs,
              'batch_size': batch_size,
              'model': model,
              'classifier': classifier,
              'criterion': criterion,
              'scheduler': scheduler,
              'optimizer': optimizer.state_dict(),
              'state_dict': model.state_dict(),
              'class_to_idx': model.class_to_idx
             }
torch.save(checkpoint, 'densenet161.pth')


In [3]:
def load_checkpoint(filepath, map_location='cpu'):
    checkpoint = torch.load('checkpoint.pth')
    model = checkpoint['model']
    model.fc = checkpoint['classifier']
    model.load_state_dict(checkpoint['state_dict'],strict=False)
    model.class_to_idx = checkpoint['class_to_idx']
    optimizer = checkpoint['optimizer']
    scheduler = checkpoint['scheduler']
    criterion = checkpoint['criterion']
    epochs = checkpoint['epochs']
    
    for param in model.parameters():
        param.requires_grad = False
        
    return model, checkpoint['class_to_idx']

device = torch.device("cuda")
model, class_to_idx = load_checkpoint('checkpoint.pth')
model

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [ ]:
def process_image(image):
    img_pil = Image.open(image)
   
    adjustments = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    img_tensor = adjustments(img_pil)
    
    return img_tensor
    
    
    # TODO: Process a PIL image for use in a PyTorch model

img = (data_dir + '/test' + '/1/' + 'image_06752.jpg')
img = process_image(img)
print(img.shape)

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

    img = Image.open(image_path)
    img = process_image(img_path)
    
    # Convert 2D image to 1D vector
    img = np.expand_dims(img, 0)
    
    
    img = torch.from_numpy(img)
    
    model.eval()
    inputs = Variable(img).to(device)
    logits = model(inputs)
    
    ps = F.softmax(logits,dim=1)
    topk = ps.cpu().topk(topk)
    
    return (e.data.numpy().squeeze().tolist() for e in topk)

In [ ]:
img_path = (data_dir + '/test' + '/1/' + 'image_06752.jpg')
probs, classes = predict(img_path, model.to(device))
print(probs)
print(classes)
flower_names = [cat_to_name[class_names[e]] for e in classes]
print(flower_names)

In [ ]:
def view_classify(img_path, prob, classes, mapping):
    ''' Function for viewing an image and it's predicted classes.
    '''
    image = Image.open(img_path)
    fig, (ax1, ax2) = plt.subplots(figsize=(6,10), ncols=1, nrows=2)
    flower_name = mapping[img_path.split('/')[-2]]
    ax1.set_title(flower_name)
    ax1.imshow(image)
    ax1.axis('off')
    
    y_pos = np.arange(len(prob))
    ax2.barh(y_pos, prob, align='center')
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(flower_names)
    ax2.invert_yaxis()  # labels read top-to-bottom
    ax2.set_title('Class Probability')
view_classify(img_path, probs, classes, cat_to_name)

In [ ]:
check_sanity()